# Player Statistics

Get the saved games

In [1]:
import json, os
gamesRawData = []
for i in os.listdir("./games"):
    if i.endswith(".json"):
        with open("./games/"+i,"r") as f:
            gamesRawData.append(json.loads(f.read()))

The rotation score is an experimental metric presentab by Doran's Lab : https://doranslab.gg/articles/location-based-champ-metrics.html

In [2]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

#Calculate the rotation score ffrom one position to another.
def getRotationScore(pos1, pos2, team):
    #Get the spawn point position as the base for the angle
    basePosition = np.array([394, 461]) if team==100 else np.array([14340, 14391])
    
    #Translate position to numpy array
    p1 = np.array([pos1["x"],pos1["y"]])
    p2 = np.array([pos2["x"],pos2["y"]])
    
    #Get the distance for both positions
    p1Distance = np.linalg.norm(p1-basePosition)
    p2Distance = np.linalg.norm(p2-basePosition)
    
    #Get the vector for both positions
    p1vec = p1 - basePosition
    p2vec = p2 - basePosition
    
    #Get the angle value of the two vectors
    cosine_angle = np.dot(p1vec,p2vec) / (np.linalg.norm(p1vec) * np.linalg.norm(p2vec))
    angle = np.arccos(cosine_angle) 
    
    #Return the rotation score, which is the mean distance multiplied by the angle value
    return (p1Distance+p2Distance)/2 * angle


#Get the player rotation score from 5 to 15 minutes
def getPlayerRotationScore(timeline, participantId, teamId):
    score = 0
    for i in range(6,16):
        score += getRotationScore(
            timeline["frames"][i]["participantFrames"][str(p["participantId"])]["position"],
            timeline["frames"][i-1]["participantFrames"][str(p["participantId"])]["position"],
            teamId
        )
    return score


A simple function to get the role of a player. As it is esport format, it is possible to get them only by the position in the team.

In [3]:
def getRole(participantId):
    if participantId in [1,6]:
        return "Toplaner"
    elif participantId in [2,7]:
        return "Jungler"
    elif participantId in [3,8]:
        return "Midlaner"
    elif participantId in [4,9]:
        return "ADC"
    elif participantId in [5,10]:
        return "Support"

Parse all the games and get the player stats

In [4]:
playersRows=[]

for game in gamesRawData:
    
    #Uncomment to filter playin games
    '''
    if game["gameCreation"]<1539122400000:
        continue
    '''
    #Count some stats of the team
    teamStats = {
        100:{"kills":0,"damagesToChampions":0,"damagesToTurrets":0,"gold":0},
        200:{"kills":0,"damagesToChampions":0,"damagesToTurrets":0,"gold":0}
    }
    for p in game["participants"]:
        teamStats[p["teamId"]]["kills"] += p["stats"]["kills"]
        teamStats[p["teamId"]]["damagesToChampions"] += p["stats"]["totalDamageDealtToChampions"]
        teamStats[p["teamId"]]["damagesToTurrets"] += p["stats"]["damageDealtToTurrets"]
        teamStats[p["teamId"]]["gold"] += p["stats"]["goldEarned"]
        
    pIdToName = {}
    #Player identities
    for p in game["participantIdentities"]:
        pIdToName[p["participantId"]] = p["player"]["summonerName"]
    
    
    for p in game["participants"]:
        #get player frame @15min
        playerFrame = game["timeline"]["frames"][15]["participantFrames"][str(p["participantId"])]
        opponentFrame = game["timeline"]["frames"][15]["participantFrames"][str((int(p["participantId"])+5)%10 if not p["participantId"]==5 else 10)]
        
        player={}
        #Player information
        player["team"], player["name"] = pIdToName[p["participantId"]].split(" ")
        player["role"] = getRole(p["participantId"])
        
        player["gameDuration"] = game["gameDuration"]
        
        #Add team stats
        player["teamKills"] = teamStats[p["teamId"]]["kills"]
        player["teamDamagesToChampions"] = teamStats[p["teamId"]]["damagesToChampions"]
        player["teamDamagesToTurrets"] = teamStats[p["teamId"]]["damagesToTurrets"]
        player["teamGold"] = teamStats[p["teamId"]]["gold"]
        
        #KDA stats
        player["kills"] = p["stats"]["kills"]
        player["deaths"] = p["stats"]["deaths"]
        player["assists"] = p["stats"]["assists"]
        
        #Damages stats
        player["damagesToChampions"] = p["stats"]["totalDamageDealtToChampions"]
        player["damagesToTurrets"] = p["stats"]["damageDealtToTurrets"]
        
        #Gold stats
        player["gold"] = p["stats"]["goldEarned"]
        player["gold@15"] = playerFrame["totalGold"]
        player["goldAdvantage@15"] = playerFrame["totalGold"] - opponentFrame["totalGold"]
        
        #Farm stats
        player["creepScore"] = p["stats"]["neutralMinionsKilled"] + p["stats"]["totalMinionsKilled"]
        player["CS@15"] = playerFrame["minionsKilled"] + playerFrame["jungleMinionsKilled"]
        player["CSAdvantage@15"] = playerFrame["minionsKilled"] + playerFrame["jungleMinionsKilled"] - (opponentFrame["minionsKilled"] + opponentFrame["jungleMinionsKilled"])
        
        #Vision score
        player["visionScore"] = p["stats"]["visionScore"]
        
        #Rotation score
        player["rotationScore"] = getPlayerRotationScore(game["timeline"],p["participantId"],p["teamId"])
        
        playersRows.append(player)

Get the list of players to a dataframe

In [5]:
import pandas as pd
dfPlayers = pd.DataFrame(playersRows)
dfPlayers.head()

,CS@15,CSAdvantage@15,assists,creepScore,damagesToChampions,damagesToTurrets,deaths,gameDuration,gold,gold@15,...,kills,name,role,rotationScore,team,teamDamagesToChampions,teamDamagesToTurrets,teamGold,teamKills,visionScore
0,149,4,4,420,9866,4281,1,2522,17929,5585,...,3,Hanabi,Toplaner,16428.238712,FW,55710,15808,78179,12,76
1,82,-23,7,190,5761,894,1,2522,12488,4454,...,0,Moojin,Jungler,39526.588737,FW,55710,15808,78179,12,59
2,151,-1,7,376,7944,778,0,2522,16153,5163,...,1,Maple,Midlaner,13964.362043,FW,55710,15808,78179,12,47
3,151,-11,2,473,29860,8545,0,2522,20881,5144,...,8,Betty,ADC,18844.323795,FW,55710,15808,78179,12,79
4,22,18,9,85,2279,1310,0,2522,10728,2974,...,0,SwordArt,Support,27854.851166,FW,55710,15808,78179,12,113


Add stats representing player share in the team

In [6]:
dfPlayers["championDamagesPart"] = dfPlayers["damagesToChampions"] / dfPlayers["teamDamagesToChampions"]
dfPlayers["turretDamagesPart"] = dfPlayers["damagesToTurrets"] / dfPlayers["teamDamagesToTurrets"]
dfPlayers["KP"] = (dfPlayers["kills"] + dfPlayers["assists"])/ dfPlayers["teamKills"]

Note that in this dataframe, each row is stats from a player from a specific game. To get average stats for each player, use the groupby function

In [7]:
dfPlayersGrouped = dfPlayers.groupby(["name","team","role"]).mean().reset_index()
dfPlayersGrouped.head()

,name,team,role,CS@15,CSAdvantage@15,assists,creepScore,damagesToChampions,damagesToTurrets,deaths,...,kills,rotationScore,teamDamagesToChampions,teamDamagesToTurrets,teamGold,teamKills,visionScore,championDamagesPart,turretDamagesPart,KP
0,Ambition,GEN,Jungler,87.000000,-16.000000,3.000000,155.000000,9018.000000,1758.000000,1.000000,...,2.000000,27721.629060,43473.000000,8047.000000,56177.000000,6.000000,45.000000,0.207439,0.218467,0.833333
1,AnDa,100,Jungler,82.000000,-1.333333,5.166667,150.000000,10903.000000,2361.833333,2.833333,...,4.833333,42966.644738,53414.166667,10672.333333,51642.500000,12.333333,45.000000,0.195632,0.194818,0.711429
2,Arce,INF,Support,22.125000,-3.875000,7.875000,60.750000,4398.000000,525.250000,3.000000,...,0.375000,20713.020397,63539.375000,11696.500000,57840.625000,10.750000,81.125000,0.068204,0.035242,0.802827
3,Attila,VIT,ADC,138.000000,5.000000,5.000000,379.500000,19667.333333,6529.500000,1.666667,...,4.000000,21056.697962,59985.500000,13797.166667,65466.333333,11.833333,70.333333,0.312489,0.450167,0.766445
4,Baolan,IG,Support,20.833333,0.750000,8.416667,44.416667,4343.500000,872.750000,2.000000,...,1.000000,21010.334817,59658.333333,15958.416667,57892.333333,13.666667,74.750000,0.075057,0.053235,0.669729


KDA is not calculated as a mean, and needs to be done aside, using sums

In [8]:
dfGroupedSummed = dfPlayers.groupby(["name","team"]).sum().reset_index()

dfPlayersGrouped["KDA"] = 0
dfPlayersGrouped.loc[dfGroupedSummed["deaths"]>0,"KDA"] = ((dfGroupedSummed.loc[dfGroupedSummed["deaths"]>0]["kills"] + dfGroupedSummed.loc[dfGroupedSummed["deaths"]>0]["assists"]) / dfGroupedSummed.loc[dfGroupedSummed["deaths"]>0]["deaths"])
#Handling the case when KDA is infinite (deaths=0)
dfPlayersGrouped.loc[dfGroupedSummed["deaths"]==0,"KDA"]=(dfGroupedSummed.loc[dfGroupedSummed["deaths"]==0]["kills"] + dfGroupedSummed.loc[dfGroupedSummed["deaths"]==0]["assists"])

As this is the summary of the stats for each player, this is the one that will be exported to csv

In [9]:
dfPlayersGrouped.to_csv("players.csv", index=False)

Some manipulation examples

Get the players with the best Kill Participation

In [10]:
dfPlayersGrouped[["name","team","role","KP"]].sort_values("KP", ascending=False).head()

,name,team,role,KP
50,Kongyue,MAD,Jungler,0.922619
92,Shernfire,DW,Jungler,0.884615
31,G4,ASC,Midlaner,0.880952
0,Ambition,GEN,Jungler,0.833333
118,XuHao,PVB,Jungler,0.826087


Get the Kill Participation by role

In [11]:
dfPlayersGrouped.groupby(["role"]).mean()[["KP"]].sort_values("KP", ascending=False)

,KP
role,
Jungler,0.714847
Support,0.683101
Midlaner,0.667739
ADC,0.663763
Toplaner,0.550085


Get the players with the best CS lead

In [12]:
dfPlayersGrouped[["name","team","role","CSAdvantage@15"]].sort_values("CSAdvantage@15", ascending=False).head()

,name,team,role,CSAdvantage@15
87,Rookie,IG,Midlaner,22.500000
113,Uzi,RNG,ADC,21.500000
79,Ray1,EDG,Toplaner,19.000000
74,Perkz,G2,Midlaner,17.238095
66,Nate,KLG,Toplaner,17.000000


Get the players with the best Gold lead

In [13]:
dfPlayersGrouped[["name","team","role","goldAdvantage@15"]].sort_values("goldAdvantage@15", ascending=False).head()

,name,team,role,goldAdvantage@15
79,Ray1,EDG,Toplaner,822.000000
113,Uzi,RNG,ADC,749.833333
91,Scout,EDG,Midlaner,747.705882
87,Rookie,IG,Midlaner,629.000000
122,Zeros,PVB,Toplaner,552.833333


Get player with the best KDA

In [14]:
dfPlayersGrouped[["name","team","role","KDA"]].sort_values("KDA", ascending=False).head()

,name,team,role,KDA
80,Rekkles,FNC,ADC,10.900000
11,Broxah,FNC,Jungler,9.909091
17,Clearlove,EDG,Jungler,9.000000
36,Hjarnan,G2,ADC,7.869565
25,Doublelift,TL,ADC,7.000000
